In [7]:
import pandas as pd
import numpy as np
import plotly.express as px
import sqlite3
import folium

In [9]:
############ DEUXIEME CARTE
############
# Création de la liste des années disponibles

annees = ['2019', '2020', '2021', '2022']
# Création de la liste des types d'objets disponibles

# Connexion à la base de données
connexion = sqlite3.connect('bdd.db')

# Lecture de la table objet
# Récupération des données de la table objet
query_objet = "SELECT * FROM objet"
df = pd.read_sql_query(query_objet, connexion)
types = df['type'].unique().tolist()

# Récupération des données de la table gare
query_gare = "SELECT * FROM gare"
df_gare = pd.read_sql_query(query_gare, connexion)

gares = df_gare.gare.unique()
dict_geo_gare = {}
for gare in gares:
    lat = df_gare.loc[df_gare['gare'] == gare, 'lat'].values[0]
    long = df_gare.loc[df_gare['gare'] == gare, 'lon'].values[0]
    dict = {'lat': lat, 'long':long}
    dict_geo_gare[gare] = dict


# Transformation de la colonne 'date' en datetime
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y')

# Filtrage par année et type d'objet
annee = st.selectbox("Sélectionnez l'année :", annees, key=2)

type_objet = st.multiselect("Sélectionnez le type d'objet :", types, default=types[:], key=4)
df_filtre = df[(df_objet['type'].isin(type_objet)) & (df_objet['date'].str[-4:] == annee)]

# Agrégation par gare
df_agg = df_filtre.groupby('gare').agg({'id': 'count', 'date': 'nunique'}).reset_index()
df_agg.rename(columns={'id': 'nombre_objets', 'date': 'nombre_jours'}, inplace=True)

# Création de la carte de Paris
paris_map = folium.Map(location=[48.864716, 2.349014], zoom_start=12)

# Ajout des barres pour chaque gare
for _, row in df_agg.iterrows():
    nom_gare = row['gare']
    lat = dict_geo_gare[nom_gare]['lat']
    long = dict_geo_gare[nom_gare]['long']
    nombre_objets = row['nombre_objets']
    nombre_jours = row['nombre_jours']
    
    popup_text = f"{nom_gare}<br>Nombre d'objets: {nombre_objets}<br>Nombre de jours: {nombre_jours}"
    folium.Marker([lat, long], popup=popup_text).add_to(paris_map)

    folium.vector_layers.Rectangle(
        bounds=((lat - 0.002, long - 0.002), (lat + 0.002, long + 0.002)),
        fill=True,
        fill_color='blue',
        fill_opacity=0.5,
        color=None,
        weight=0,
        tooltip=f"Nombre de jours: {nombre_jours}",
        ).add_to(paris_map)

    folium.vector_layers.Rectangle(
        bounds=((lat - 0.002, long + 0.002), (lat + 0.002, long + 0.006 * nombre_objets)),
        fill=True,
        fill_color='red',
        fill_opacity=0.5,
        color=None,
        weight=0,
        tooltip=f"Nombre d'objets: {nombre_objets}",
        ).add_to(paris_map)

# Affichage de la carte
paris_map

NameError: name 'df_objet' is not defined